# Índice
* [Scraping](../scraper/Web-scraping.ipynb)
* [Limpieza y Feature engineering](../limpieza/Limpieza-y-exploracion-de-datos-no-estructurados-con-spark.ipynb)
* [Modelado](#Modelado)

# Modelado

## Objetivo

**Predecir precios de venta de casas en San Francisco**

- Los **datos**: Zillow. Información de casas en diferentes zonas, el precio y fecha en que se vendieron y el precio que se puso en los anuncios.
- Variable a predecir: **lastsoldprice**.
- El dataset tbn incluye zestimate: el valor estimado de venta que calculó Zillow pero no lo vamos a tomar en cuenta.
- Haremos un análisis exploratorio brevísimo, una regresión lineal y un bosque aleatorio.

In [ ]:
# Importamos librerías para trabajar
%matplotlib inline
import numpy as np
import pandas as pd
np.random.seed(66) # Importante para hacer los resultados reproducibles!!!
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
# Leemos el csv de datos
sf = pd.read_csv('final_data.csv')
sf.head()

In [ ]:
# Contamos renglones y columnas
sf.shape

In [ ]:
# Tiramos algunas columnas que no necesitaremos para modelar
# info,z_address,usecode,zipcode: vamos a considerar sólo neighborhood para indicarnos dónde está la casa.
# recordemos que zestimate es el valor estimado por Zillow. No lo queremos incluir en el modelo y tbn lo quitamos.
sf.drop(sf.columns[[0, 2, 3, 15, 17, 18]], axis=1, inplace=True)
sf.head()

In [ ]:
sf.info()

In [ ]:
# Vemos que la variable zindexvalue tiene comas! es texto!!! No puede entrar así al modelo. Casteamos
sf['zindexvalue'] = sf['zindexvalue'].str.replace(',', '')
sf['zindexvalue'] = sf['zindexvalue'].convert_objects(convert_numeric=True)

In [ ]:
# Estadísticos resumen de  variables numéricas
sf.describe()

In [ ]:
#Histogramas para ver qué hay en cada columna numérica
%matplotlib inline
import matplotlib.pyplot as plt
sf.hist(bins=50, figsize=(20,15))
# Podemos guardar la gráfica en un .png para poder usarla en presentaciones o pdfs
plt.savefig("attribute_histogram_plots")
plt.show()

# Variables tienen diferente escala
# Sesgo puede dificultar que los modelos funcionen; cosas con distribuciones normales funcionan mejor

In [ ]:
# Tenemos latitud y longitud. Podemos pintar un scatterplot tal cual
sf.plot(kind="scatter", x="longitude", y="latitude", alpha=0.2)
plt.savefig('map1.png')

In [ ]:
# Podemos pintar puntos en relación con el precio, que es lo que queremos predecir
sf.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, figsize=(10,7),
    c="lastsoldprice", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
plt.savefig('map2.png')
# OJO! Podemos tener mapas más bonitos con plotly y mapbox

In [ ]:
# Correlaciones del lastsoldprice contra todo

corr_matrix = sf.corr()
corr_matrix["lastsoldprice"].sort_values(ascending=False)

# finishedsqft y número de bathrooms tienen corr positiva y grande con lastsoldprice
# Hay corr negativa con yearbuilt pero chica
# Entre más cercano a cero, menor correlación linear

In [ ]:
# Elegimos las variables que se ven más correlacionadas con lastsoldprice y pintamos un scatterplot
from pandas.tools.plotting import scatter_matrix

attributes = ["lastsoldprice", "finishedsqft", "bathrooms", "zindexvalue"]
scatter_matrix(sf[attributes], figsize=(12, 8))
plt.savefig('matrix.png')

In [ ]:
# Lo que mejor se ve es lastsoldprice y finishedsqft
sf.plot(kind="scatter", x="finishedsqft", y="lastsoldprice", alpha=0.5)
plt.savefig('scatter.png')

In [ ]:
# OJO, cada casa tiene diferentes metros cuadradas. Casas más grandes seguro son más caras.
# Para poder comparar diferentes casas, necesitamos normalizar: precios por metro cuadrado
sf['price_per_sqft'] = sf['lastsoldprice']/sf['finishedsqft']
corr_matrix = sf.corr()
corr_matrix["lastsoldprice"].sort_values(ascending=False)

In [ ]:
# La variable nueva (price_per_sqft) no tiene muy buena correlación con la que queremos predecir (lastsoldprice)
# Quizá si agrupamos por neighbourhood salgan mejor los modelos
len(sf['neighborhood'].value_counts())

In [ ]:
# Tiene sentido que el precio por metro cuadrado varíe por zona
# veamos cuántas casas hay por neighborhood
freq = sf.groupby('neighborhood').count()['address']
# Y veamos el precio promedio para cada una de ellas
mean = sf.groupby('neighborhood').mean()['price_per_sqft']
# creamos un dataset nuevo
cluster = pd.concat([freq, mean], axis=1)
cluster['neighborhood'] = cluster.index
cluster.columns = ['freq', 'price_per_sqft','neighborhood']
cluster.head()

In [ ]:
# Y veamos resumen de las variables
cluster.describe()

In [ ]:
# Observamos la media de precios por metro cuadrado: 756.
# Y observamos la media de casas en cada colonia: 123
# Podemos dividir las casas en 4 grupos, considerando ambas variables por arriba y por abajo de la media.
# Spoiler alert: en las baratas no sirve de mucho tener grupos diferenciados según el número de casas en cada colonia
# Así que sólo hacemos 3 clusters:

# Podemos hacer un primer cluster de casas baratas
cluster1 = cluster[cluster.price_per_sqft < 756]
cluster1.index

In [ ]:
# Otro cluster de casas caras y en colonias con pocas casas
cluster_temp = cluster[cluster.price_per_sqft >= 756]
cluster2 = cluster_temp[cluster_temp.freq <123]
cluster2.index

In [ ]:
# Un tercero para casas caras en colonias con muchas casas
cluster3 = cluster_temp[cluster_temp.freq >=123]
cluster3.index

In [ ]:
# Para usar esos clusters, hay que agregar una columna indicando a cuál grupo pertenece cada casa
def get_group(x):
    if x in cluster1.index:
        return 'low_price'
    elif x in cluster2.index:
        return 'high_price_low_freq'
    else:
        return 'high_price_high_freq'
# cluster.index es la neighborhood
# agregamos al dataset original el grupo, haciendo un apply de la función que acabamos de definir
sf['group'] = sf.neighborhood.apply(get_group)
sf.head()

In [ ]:
# En los clusters estamos incluyendo información de ubicación. Podemos matar columnas que ya no sirven:
sf.drop(sf.columns[[0, 4, 6, 7, 8, 13]], axis=1, inplace=True)
sf = sf[['bathrooms', 'bedrooms', 'finishedsqft', 'totalrooms', 'usecode', 'yearbuilt','zindexvalue', 'group', 'lastsoldprice']]
sf.head()

In [ ]:
sf.info()

In [ ]:
# Seguimos teniendo columnas no numéricas: usecode y group
# Son categóricas. Necesitamos tener indicadoras.
# Dividimos en variables explicativas y variable a predecir
X = sf[['bathrooms', 'bedrooms', 'finishedsqft', 'totalrooms', 'usecode', 'yearbuilt', 
         'zindexvalue', 'group']]
Y = sf['lastsoldprice']

# Usamos una función de pandas para sacar indicadoras y las agregamos al dataset
n = pd.get_dummies(sf.group)
X = pd.concat([X, n], axis=1)

m = pd.get_dummies(sf.usecode)
X = pd.concat([X, m], axis=1)

# Tiramos las variables categóricas originales porque ya las cambiamos por indicadoras.
drops = ['group', 'usecode']
X.drop(drops, inplace=True, axis=1)

X.head()

In [ ]:
# Ahora sí a entrenar el modelo
from sklearn.cross_validation import train_test_split

# dividimos en train y test, con 70%-30% para cada uno
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

# Ojo con la notación que permite asignar en 4 objetos el resultado de train_test_split

In [ ]:
# Regresión lineal
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

In [ ]:
# R cuadrada: qué porciento de la variabilidad en Y se explica vía X
y_pred = regressor.predict(X_test)
print('R cuadrada": %.4f' % regressor.score(X_test, y_test))

In [ ]:
# Qué tan cerca están las predicciones del valor real? raíz del error cuadrático medio
import numpy as np
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_pred, y_test)
lin_rmse = np.sqrt(lin_mse)
print('El modelo hace predicciones que difieren a lo más %.4f del precio real' % lin_rmse)

In [ ]:
# En valor absoluto, qué tan lejos estamos en promedio?
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(y_pred, y_test)
print('Las predicciones del modelo están, en promedio a %.4f del precio real' % lin_mae)

In [ ]:
# Ahora un random forest
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(X_train, y_train)

In [ ]:
# R cuadrada: qué porciento de la variabilidad en Y se explica vía X
print('R cuadrada del random forest: %.4f' % forest_reg.score(X_test, y_test))

In [ ]:
y_pred = forest_reg.predict(X_test)
forest_mse = mean_squared_error(y_pred, y_test)
forest_rmse = np.sqrt(forest_mse)
print('El random forest hace predicciones que difieren a lo más %.4f del precio real' % forest_rmse)

In [ ]:
# Es importante poder explicar el modelo.
# En una regresión, los coeficientes dan sentido de la importancia. En un bosque aleatorio?

feature_labels = np.array(['bathrooms', 'bedrooms', 'finishedsqft', 'totalrooms', 'yearbuilt', 'zindexvalue', 
                           'high_price_high_freq', 'high_price_low_freq', 'low_price', 'Apartment', 'Condominium', 'Cooperative', 
                          'Duplex', 'Miscellaneous', 'Mobile', 'MultiFamily2To4', 'MultiFamily5Plus', 'SingleFamily', 
                           'Townhouse'])
importance = forest_reg.feature_importances_
feature_indexes_by_importance = importance.argsort()
for index in feature_indexes_by_importance:
    print('{}-{:.2f}%'.format(feature_labels[index], (importance[index] *100.0)))